# SVM Classifier (Color Extraction)

## Initiation

https://github.com/tsunghao-huang/SVM-Fruit-Image-Classifier/blob/master/main.py
http://localhost:8888/notebooks/RTC:Downloads/Shanmukh_Classification.ipynb

In [1]:
import os
import cv2
import imutils
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection

In [2]:
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [3]:
# Define data

# datadir= r"C:\\Users\SMSK_Narinda\Desktop\Research\dataset" 
datadir= r"C:\Users\SMSK_Narinda\Desktop\Research\data_split_augmented\test"

In [4]:
Categories=[]
# img_size = 100

In [5]:
subfolders= [os.path.join(datadir, d) for d in os.listdir(datadir) if os.path.isdir(os.path.join(datadir, d))]

In [6]:
for label in os.listdir(datadir):
    Categories.append(label)

print(Categories)

['Acne_and_Rosacea_Photos', 'Actinic_Keratosis_Basal_Cell_Carcinoma_and_other_Malignant_Lesions', 'Atopic_Dermatitis_Photos', 'Bullous_Disease_Photos', 'Cellulitis_Impetigo_and_other_Bacterial_Infections', 'Eczema_Photos', 'Exanthems_and_Drug_Eruptions', 'Hair_Loss_Photos_Alopecia_and_other _Hair_Diseases', 'Herpes_HPV_other_STDs_Photos', 'Light_Diseases_and_Disorders_of_Pigmentation', 'Lupus_and_other_Connective_Tissue_diseases', 'Melanoma_Skin_Cancer_Nevi_and_Moles', 'Nail_Fungus_and_other_Nail_Disease', 'Poison_Ivy_Photos_and_other_Contact_Dermatitis', 'Psoriasis_pictures_Lichen_Planus_and_related_diseases', 'Scabies_Lyme_Disease_and_other_Infestations_and_Bites', 'Seborrheic_Keratoses_and_other_Benign_Tumors', 'Systemic_Disease', 'Tinea_Ringworm_Candidiasis_and_other_Fungal_Infections', 'Urticaria_Hives', 'Vascular_Tumors', 'Vasculitis_Photos', 'Warts_Molluscum_and_other_Viral_Infections']


In [7]:
label_to_index = {category_name: index for index, category_name in enumerate(Categories)}
print(label_to_index)

{'Acne_and_Rosacea_Photos': 0, 'Actinic_Keratosis_Basal_Cell_Carcinoma_and_other_Malignant_Lesions': 1, 'Atopic_Dermatitis_Photos': 2, 'Bullous_Disease_Photos': 3, 'Cellulitis_Impetigo_and_other_Bacterial_Infections': 4, 'Eczema_Photos': 5, 'Exanthems_and_Drug_Eruptions': 6, 'Hair_Loss_Photos_Alopecia_and_other _Hair_Diseases': 7, 'Herpes_HPV_other_STDs_Photos': 8, 'Light_Diseases_and_Disorders_of_Pigmentation': 9, 'Lupus_and_other_Connective_Tissue_diseases': 10, 'Melanoma_Skin_Cancer_Nevi_and_Moles': 11, 'Nail_Fungus_and_other_Nail_Disease': 12, 'Poison_Ivy_Photos_and_other_Contact_Dermatitis': 13, 'Psoriasis_pictures_Lichen_Planus_and_related_diseases': 14, 'Scabies_Lyme_Disease_and_other_Infestations_and_Bites': 15, 'Seborrheic_Keratoses_and_other_Benign_Tumors': 16, 'Systemic_Disease': 17, 'Tinea_Ringworm_Candidiasis_and_other_Fungal_Infections': 18, 'Urticaria_Hives': 19, 'Vascular_Tumors': 20, 'Vasculitis_Photos': 21, 'Warts_Molluscum_and_other_Viral_Infections': 22}


## Preprocessing

In [8]:
# features = []
# labels = []
flat_data_arr=[]
target_arr=[]

In [9]:
for i in range (1,2):
    print("[INFO] ITERATION", i)
    for classes in subfolders:
        label = classes.split(os.path.sep)[-1]

        # Get the label (category name) from the subfolder
        print("Processing for label " + label)

        # Iterate over the images in the current subfolder        
        for filename in os.scandir(classes):
            if filename.is_file():
                # Get the path of the image file
                image_path = filename.path

                # Read and resize the image
                image = cv2.imread(image_path)
                image = cv2.resize(image, (150, 150))
                 
                # # Ekstrak fitur dari gambar dan histogram warna
                # # pixels = image_2_feature_vector(image)
                # hist = extract_color_histogram(image)
                
                # Append the flattened image data and numerical label to their respective lists
                flat_data_arr.append(image.flatten())
                target_arr.append(label_to_index[label])
    
                # Menambahkan fitur dan label ke dalam list yang sesuai
                # rawImages.append(pixels)
                # features.append(hist)
                # labels.append(label)
    print("[STOP] \n")

[INFO] ITERATION 1
Processing for label Acne_and_Rosacea_Photos
Processing for label Actinic_Keratosis_Basal_Cell_Carcinoma_and_other_Malignant_Lesions
Processing for label Atopic_Dermatitis_Photos
Processing for label Bullous_Disease_Photos
Processing for label Cellulitis_Impetigo_and_other_Bacterial_Infections
Processing for label Eczema_Photos
Processing for label Exanthems_and_Drug_Eruptions
Processing for label Hair_Loss_Photos_Alopecia_and_other _Hair_Diseases
Processing for label Herpes_HPV_other_STDs_Photos
Processing for label Light_Diseases_and_Disorders_of_Pigmentation
Processing for label Lupus_and_other_Connective_Tissue_diseases
Processing for label Melanoma_Skin_Cancer_Nevi_and_Moles
Processing for label Nail_Fungus_and_other_Nail_Disease
Processing for label Poison_Ivy_Photos_and_other_Contact_Dermatitis
Processing for label Psoriasis_pictures_Lichen_Planus_and_related_diseases
Processing for label Scabies_Lyme_Disease_and_other_Infestations_and_Bites
Processing for lab

In [10]:
print("[INFO] Processed {}/{}".format(len(target_arr), len(subfolders)))

[INFO] Processed 7488/23


In [11]:
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)

In [12]:
print("[INFO] flat_data matrix: {:.2f} MB".format(
	flat_data.nbytes / (1024 * 1000.0)))
print("[INFO] target matrix: {:.2f} MB".format(
	target.nbytes / (1024 * 1000.0)))
print("flat_data shape ", flat_data.shape)
print("target shape ", target.shape)

[INFO] flat_data matrix: 493.59 MB
[INFO] target matrix: 0.03 MB
flat_data shape  (7488, 67500)
target shape  (7488,)


In [13]:
df=pd.DataFrame(flat_data)

In [15]:
# x_train, x_test, y_train, y_test = train_test_split(
#     x, y,test_size=0.30,random_state=42,stratify=y)

# print('Splitted Successfully')

x_train, x_test, y_train, y_test = train_test_split(
    flat_data, target, test_size=0.30, random_state=42, stratify=target)
print('Splitted Successfully')

Splitted Successfully


## Training Model

DASAR svm
https://stackoverflow.com/questions/19634531/support-vector-machine-algorithm-for-image-classification

In [72]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [73]:
model = SVC(max_iter=5000)
model.fit(x_train, y_train)

SVC(max_iter=5000)

In [74]:
# Prediksi dan evaluasi
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6048064085447263


#### cara #2

In [82]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [83]:
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf2 = clf.fit(x_train, y_train)

In [84]:
prediction2 = clf2.predict(x_test)

In [85]:
acc2 = accuracy_score(y_test, prediction2)

#### cara #3

In [16]:
from sklearn.model_selection import GridSearchCV

# Define SVM model
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf3 = GridSearchCV(svc, parameters)
model = clf3.fit(x_train, y_train)

In [17]:
prediction3 = clf3.predict(x_test)
print(clf3.predict(x_test))

[11 12 12 ...  2 14 17]


In [18]:
acc3 = accuracy_score(y_test, prediction3)

#### cara #4

linearSVC
linearSVR (regrsi
NuSVC
NuSVR (regresi
OneClassSVM
SVC
SVR (regresi
l1_min_c
hasi; linear aja = 1.00
tapi kita ulang lagi pake x_train dkk

In [90]:
clf4 = SVC(kernel='linear')
clf4 = clf4.fit(x_train, y_train)

In [91]:
prediction4 = clf.predict(x_test)
print(clf.predict(x_test))

[11 12 12 ...  5 15 17]


In [92]:
acc4 = accuracy_score(y_test, prediction4)

In [97]:
print(acc2)
print(acc3)
print(acc4)

0.6048064085447263
0.7169559412550067
0.6048064085447263


## Evaluating Model; Model Summary

In [ ]:
# Cross Validation for model
model = clf3
scores = cross_val_score(
    model, x_train, y_train, 
    cv=5, scoring='accuracy')

In [ ]:
print("[INFO] Cross Validation Model Score : %0.2f (+/- %0.2f)" 
      % (scores.mean()*100, scores.std()*2))
print("[RESULT] Score mean: ", scores.mean()*100, '%')

In [ ]:
# Additional statistic report of the model
# Source report :
# https://www.datasklr.com/select-classification-methods/k-nearest-neighbors

# Reshape the arrays
testLabels_encoded = y_test.reshape(-1, 1)
prediction_encoded = prediction_encoded.reshape(-1, 1)

# Now you can use the reshaped arrays with sklearn metrics functions
print('Average Precision : ', metrics.average_precision_score(testLabels_encoded, prediction_encoded)*100, '%') 
print('F1 Score : ', metrics.f1_score(testLabels_encoded, prediction_encoded, average='weighted')*100, '%')
print('Precision : ', metrics.precision_score(testLabels_encoded, prediction_encoded, average='weighted')*100, '%')
print('Recall : ', metrics.recall_score(testLabels_encoded, prediction_encoded, average='weighted')*100, '%')

## Classification Report

In [ ]:
# Describe unique classes in dataset
classes = []
for unique_class in subfolders:
    # Get label from the subfolder name 
    label = unique_class.split(os.path.sep)[-1]
    classes.append(label)

# Describe classification report
print(classification_report(testLabels, prediction, target_names=classes))

## Confusion Matrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        testFeat,
        testLabels,
        display_labels=classes,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

In [ ]:
cm_df = pd.DataFrame(cf_matrix,
                     index =classes, 
                     columns = classes)

#Plotting the confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(testLabels, prediction)
print(cf_matrix)

In [ ]:
cf_matrix = confusion_matrix(y_test, predicted_classes)

plt.figure(figsize=(10, 7))
sns.heatmap(cf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

## Save Model

In [19]:
pickle.dump(model,open('SVM_71_Classifier.p','wb'))
print("[INFO] Model is saved successfully")

[INFO] Model is saved successfully


## Testing Model

In [ ]:
# Load model
model = pickle.load(open('SVM_71_Classifier.p','rb'))

In [ ]:
# Input example image
# Source contoh gambar : "C:\Users\SMSK_Narinda\Downloads\089056200_1612266979-Melanoma.jpg"
# https://www.klikdokter.com/info-sehat/kanker/perbedaan-kanker-kulit-melanoma-dan-karsinoma

path = input('Enter URL of Image')

In [ ]:
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)

In [1]:
prediction = ['Scabies Lyme Disease and other Infestations and Bites']

In [2]:
isinstance(prediction, list)

True